# Natural Language Processing (688 only) [35pts]
## Introduction

In this problem you will develop two techniques for analyzing free text documents: a bag of words approach based upon creating a TFIDF matrix, and an n-gram language model.

You'll be applying your models to the text from the Federalist Papers.  The Federalist papers were a series of essay written in 1787 and 1788 by Alexander Hamilton, James Madison, and John Jay (they were published anonymously at the time), that promoted the ratification of the U.S. Constitution.  If you're curious, you can read more about them here: https://en.wikipedia.org/wiki/The_Federalist_Papers . They are a particularly interesting data set, because although the authorship of most of the essays has been long known since around the deaths of Hamilton and Madison, there was still some question about the authorship of certain articles into the 20th century.  You'll use document vectors and language models to do this analysis for yourself.

## The dataset

You'll use a copy of the Federalist Papers downloaded from Project Guttenberg, available here: http://www.gutenberg.org/ebooks/18 .  Specifically, the "pg18.txt" file included with the homework is the raw text downloaded from Project Guttenberg.  To ensure that everyone starts with the exact same corpus, we are providing you the code to load and tokenize this document, as given below.

In [1]:
import re

def load_federalist_corpus(filename):
    """ Load the federalist papers as a tokenized list of strings, one for each eassay"""
    with open(filename, "rt") as f:
        data = f.read()
    papers = data.split("FEDERALIST")
    
    # all start with "To the people of the State of New York:" (sometimes . instead of :)
    # all end with PUBLIUS (or no end at all)
    locations = [(i,[-1] + [m.end()+1 for m in re.finditer(r"of the State of New York", p)],
                 [-1] + [m.start() for m in re.finditer(r"PUBLIUS", p)]) for i,p in enumerate(papers)]
    papers_content = [papers[i][max(loc[1]):max(loc[2])] for i,loc in enumerate(locations)]

    # discard entries that are not actually a paper
    papers_content = [p for p in papers_content if len(p) > 0]

    # replace all whitespace with a single space
    papers_content = [re.sub(r"\s+", " ", p).lower() for p in papers_content]

    # add spaces before all punctuation, so they are separate tokens
    punctuation = set(re.findall(r"[^\w\s]+", " ".join(papers_content))) - {"-","'"}
    for c in punctuation:
        papers_content = [p.replace(c, " "+c+" ") for p in papers_content]
    papers_content = [re.sub(r"\s+", " ", p).lower().strip() for p in papers_content]
    
    authors = [tuple(re.findall("MADISON|JAY|HAMILTON", a)) for a in papers]
    authors = [a for a in authors if len(a) > 0]
    
    numbers = [re.search(r"No\. \d+", p).group(0) for p in papers if re.search(r"No\. \d+", p)]
    
    return papers_content, authors, numbers
    
    

In [3]:
# AUTOLAB_IGNORE_START
papers, authors, numbers = load_federalist_corpus("pg18.txt")
l_hanmilton=[]
for x in xrange(len(papers)):
    if authors[x][0]=='HAMILTON' and len(authors[x])==1:
        l_hanmilton.append(papers[x])
print len(l_hanmilton)
#print papers
# AUTOLAB_IGNORE_STOP

52


You're welcome to dig through the code here if you're curious, but it's more important that you look at the objects that the function returns.  The `papers` object is a list of strings, each one containing the full content of one of the Federalist Papers.  All tokens (words) in the text are separated by a single space (this includes some puncutation tokens, which have been modified to include spaces both before and after the punctuation. The `authors` object is a list of lists, which each list contains the author (or potential authors) of a given paper.  Finally the `numbers` list just contains the number of each Federalist paper.  You won't need to use this last one, but you may be curious to compare the results of your textual analysis to the opinion of historians.

## Q1: Bag of words, and TFIDF [6+6pts]

In this portion of the question, you'll use a bag of words model to describe the corpus, and write routines to build a TFIDF matrix and a cosine similarity function.  Specifically, you should first implement the TFIDF function below.  This should return a _sparse_ TFIDF matrix (as for the Graph Library assignment, make sure to directly create a sparse matrix using `scipy.sparse.coo_matrix()` rather than create a dense matrix and then convert it to be sparse).

Important: make sure you do _not_ include the empty token `""` as one of your terms. 

In [9]:
import collections # optional, but we found the collections.Counter object useful
import scipy.sparse as sp
import numpy as np
import re
import string
import math
def tfidf(docs):
    """
    Create TFIDF matrix.  This function creates a TFIDF matrix from the
    docs input.

    Args:
        docs: list of strings, where each string represents a space-separated
              document
    
    Returns: tuple: (tfidf, all_words)
        tfidf: sparse matrix (in any scipy sparse format) of size (# docs) x
               (# total unique words), where i,j entry is TFIDF score for 
               document i and term j
        all_words: list of strings, where the ith element indicates the word
                   that corresponds to the ith column in the TFIDF matrix
    """
    all_words = []
    data = []
    row = []
    col = []
    rowcount = 0
    listDic = []
    for doc in docs:
        
        dic = collections.Counter(doc.split(" "))
        listDic.append(dic)
        #print dic
        for key, value in dic.iteritems():
            if key not in all_words:
                all_words.append(key)
                data.append(value)
                col.append(len(all_words)-1)
                row.append(rowcount)
            else:
                data.append(value)
                col.append(all_words.index(key))
                row.append(rowcount)
        rowcount+=1
   
    tf = sp.coo_matrix((data, (row, col)), shape=(rowcount,len(all_words)))
    idf=[]
    numDoc = rowcount
    for word in all_words:
        docWithWord = 0
        for dic in listDic:
            if word in dic:
                docWithWord+=1
        
        idf.append(math.log(float(numDoc)/float(docWithWord)))
    index = 0
    col2=[]
    row2=[]
    data2=[]
    
    for index in xrange(len(data)):
        if idf[col[index]]!=0:
            data2.append(data[index] * idf[col[index]])
            row2.append(row[index])
            col2.append(col[index])
    tfidf = sp.coo_matrix((data2, (row2, col2)), shape=(rowcount,len(all_words)))
    return tfidf,all_words
### AUTOLAB_IGNORE_START
    print idf
    print all_words
    print data
    
    print tf.todense()
    print tfidf.todense()
### AUTOLAB_IGNORE_STOP
    pass

Our version results the following result (just showing the type, size, and # of non-zero elements):

    <86x8686 sparse matrix of type '<type 'numpy.float64'>'
        with 57607 stored elements in Compressed Sparse Row format>
     
For testing, you can also run the algorithm on the following "data set" from class:

In [10]:
### AUTOLAB_IGNORE_START
data = [
    "the goal of this lecture is to explain the basics of free text processing",
    "the bag of words model is one such approach",
    "text processing via bag of words"
]

X_tfidf, words = tfidf(data)
print X_tfidf.todense()
print words

### AUTOLAB_IGNORE_STOP

[[ 1.09861229  1.09861229  1.09861229  0.          0.40546511  0.40546511
   1.09861229  1.09861229  0.40546511  1.09861229  0.81093022  1.09861229
   0.          0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.40546511  0.          0.
   0.          0.          0.          0.40546511  0.          1.09861229
   0.40546511  0.40546511  1.09861229  1.09861229  1.09861229  0.        ]
 [ 0.          0.          0.          0.          0.          0.40546511
   0.          0.          0.40546511  0.          0.          0.          0.
   0.40546511  0.40546511  0.          0.          0.          1.09861229]]
['basics', 'goal', 'this', 'of', 'is', 'processing', 'free', 'to', 'text', 'lecture', 'the', 'explain', 'one', 'bag', 'words', 'such', 'approach', 'model', 'via']


For our implementation, this returns the following output:

    [[ 0.          0.          1.09861229  1.09861229  0.          1.09861229
       0.          0.40546511  0.40546511  1.09861229  0.          1.09861229
       0.          0.          0.40546511  1.09861229  0.81093022  0.
       1.09861229]
     [ 1.09861229  1.09861229  0.          0.          0.40546511  0.          0.
       0.40546511  0.          0.          1.09861229  0.          0.
       0.40546511  0.          0.          0.40546511  1.09861229  0.        ]
     [ 0.          0.          0.          0.          0.40546511  0.          0.
       0.          0.40546511  0.          0.          0.          1.09861229
       0.40546511  0.40546511  0.          0.          0.          0.        ]]
    ['model', 'such', 'basics', 'goal', 'bag', 'this', 'of', 'is', 'processing', 'free', 'one', 'to', 'via', 'words', 'text', 'lecture', 'the', 'approach', 'explain']

Next, implement the following simply function that takes the X_tfidf matrix (though it could also take simple term frequency matrices, etc), and compute a matrix of all pair-wise cosine similarities.

In [68]:
import numpy
def cosine_similarity(X):
    """
    Return a matrix of cosine similarities.
    
    Args:
        X: sparse matrix of TFIDF scores or term frequencies
    
    Returns:
        M: dense numpy array of all pairwise cosine similarities.  That is, the 
           entry M[i,j], should correspond to the cosine similarity between the 
           ith and jth rows of X.
    """
    docNum = X.shape[0]
    a = numpy.zeros(shape=(docNum,docNum))
    for ii in xrange(docNum):
        for jj in xrange(docNum):
            i = X.getrow(ii)
            j = X.getrow(jj)
            a[ii][jj]=i.multiply(j).sum()/(math.sqrt(i.multiply(i).sum())*(math.sqrt(j.multiply(j).sum())))
            #print i.multiply(i)
            #print i.sum()
            #print (math.sqrt(i.multiply(i).sum())*(math.sqrt(j.multiply(j).sum())))
    return a
    pass

In [69]:
#print cosine_similarity(X_tfidf)

[[ 1.          0.06796739  0.07771876]
 [ 0.06796739  1.          0.10281225]
 [ 0.07771876  0.10281225  1.        ]]


If you apply this function to the example from class:

    M = cosine_similarity(X_tfidf)
    
we get the result presented in the slides:

    [[ 1.          0.06796739  0.07771876]
     [ 0.06796739  1.          0.10281225]
     [ 0.07771876  0.10281225  1.        ]]

Finally, use this model to analyze potential authorship of the unknown Federalist Papers.  Specifically, compute the average cosine similarity between all the _known_ Hamilton papers and all the _unknown_ papers (and similarly between known Madison and unknown, and Jay and unknown).  Populate the following variables with these averages.  As a quick check, our value for the `jay_mean_cosine_similarity` (and we know definitively that the unknown papers were _not_ written by Jay), is 0.064939.

In [ ]:
hamilton_mean_cosine_similarity = None 
madison_mean_cosine_similarity = None
jay_mean_cosine_similarity = None

## Q2: N-gram language models [0+10+13pts]

In this question, you will implement an n-gram model to be able to model the language used in the Federalist Papers in a more structured manner than the simple bag of words approach.  You will fill in the following class:

In [4]:
import math
class LanguageModel:
    def __init__(self, docs, n):
        """
        Initialize an n-gram language model.
        
        Args:
            docs: list of strings, where each string represents a space-separated
                  document
            n: integer, degree of n-gram model
        """
        self.n = n
        self.counts={}
        self.count_sums={}
        self.words =  set()
        for doc in docs:
            doc = doc.split(" ")
            wordsDoc = set(doc)
            self.words = self.words.union(wordsDoc)
            for i in xrange(n-1,len(doc)):
                prefix = ' '.join(doc[i-n+1:i])
                if prefix not in self.counts:
                    self.counts[prefix]={}
                    self.counts[prefix][doc[i]]=1
                    self.count_sums[prefix]=1
                else:
                    self.count_sums[prefix]+=1
                    if doc[i] not in self.counts[prefix]:
                        self.counts[prefix][doc[i]]=1
                        
                    else:
                        self.counts[prefix][doc[i]]+=1
                index = i

            
        #print len(self.words)
        pass
    
    def perplexity(self, text, alpha=1e-3):
        """
        Evaluate perplexity of model on some text.
        
        Args:
            text: string containing space-separated words, on which to compute
            alpha: constant to use in Laplace smoothing
            
        Note: for the purposes of smoothing, the dictionary size (i.e, the D term)
        should be equal to the total number of unique words used to build the model
        _and_ in the input text to this function.
            
        Returns: perplexity
            perplexity: floating point value, perplexity of the text as evaluted
                        under the model.
        """
        sumLog = 0
        n =self.n
        text = text.split(" ")
        N = len(text)
        wordSet = set(text)
        D = len(self.words.union(wordSet))
        #print D
        for i in xrange(n-1,N):
            prefix = ' '.join(text[i-n+1:i])
            #print
            #D = len(self.counts[prefix])
            if prefix in self.counts:
                if text[i] in self.counts[prefix]:
                    prob = (self.counts[prefix][text[i]]+alpha)/(self.count_sums[prefix]+alpha*D)
                else:
                    prob = (alpha)/(self.count_sums[prefix]+alpha*D)
            else:
                prob = (alpha)/(alpha*D)
            #print prob
            sumLog+= math.log(prob,2)
        return math.pow(2,-sumLog/(N-n+1))
        pass
        
    def sample(self, k):
        """
        Generate a random sample of k words.
        
        Args:
            k: integer, indicating the number of words to sample
            
        Returns: text
            text: string of words generated from the model.
        """
        import random
        #print self.count_sums
        #s = ''.join(k*v for k,v in self.count_sums.items())
        txt =''
        prefix = random.choice([j for j in self.count_sums for x in xrange(self.count_sums[j])])
        #print prefix
        #print self.counts[prefix]
        txt+=prefix
        #print k
        for i in xrange(int(k)):
            if prefix in self.counts:
                newWord = random.choice([j for j in self.counts[prefix] for x in xrange(self.counts[prefix][j])])
                txt+=' '+newWord
                prefix =" ".join(prefix.split(" ")[1:]) +' '+newWord
            else:
                prefix = random.choice([j for j in self.count_sums for x in xrange(self.count_sums[j])])
                txt+=' '+prefix
        #print self.counts.iterkeys().next() 
        #for i in xrange()
        #print txt
        #print len(txt)
        return txt
        pass
### AUTOLAB_IGNORE_START
data = [
    "the goal of this lecture is to explain the basics of free text processing",
    "the bag of words model is one such approach",
    "text processing via bag of words"
]
#LanguageModel(data,3).count_sums

l =LanguageModel(l_hanmilton,3)
l.sample(50)
#l.counts["privilege of"]
# print l.counts["of the"]
# print l.perplexity(papers[0])
### AUTOLAB_IGNORE_STOP

'such regulations , on the score of usurpation cannot be safely placed , of connecticut or great britain it is warrantable to apprehend danger from the difference in the senate to his complaisance for those who are concerned to the quantity and extent of the press shall be convicted of treason shall'

important a
50
important a support , till they arrive to such a majority in the other of them . though it does not follow that the advantage of permanency far less probable that , pursuant to the essentiality of the meditated preference ? are not sufficiently definite to preclude standing armies and the hazard


### Part A: Initializing the language model

First, implement the `__init__()` function in the `LanguageModel` class.  You can store the information any way you want, but we did this by building a two-level dictionary (in fact, we used the `collections.defaultdict` class, but this only make a few things a little bit shorter ... you are free to use it or not) `self.counts`, where the first key refers to the previous $n-1$ tokens, and the second key refers to the $n$th token, and the value simply stores the count of the number of times this combination was seen.  For ease of use in later function, we also created a `self.count_sums`, which contained the number of total times each $n-1$ combination was seen.

For example, letting `l_hamilton` be a LanguageModel object built just from all the known Hamilton papers and with `n = 3`, the following varibles are populated in the object:

    l_hamilton.counts["privilege of"] = {'being': 1, 'originating': 1, 'paying': 1, 'the': 1}
    l_hamilton.count_sums["privilege of"] = 4
    
We also build a `self.dictionary` variable, which is just a `set` object containing all the unique words in across the entire set of input document.

### Part B: Computing perplexity

Next, implement the `perplexity()` function, which takes a text sample and computes the perplexity of this sample under the model.  Use the formula for perplexity from the class nodes (which is actually not exact, since it only so, being careful to not multiply togther probabilites that get too small (hint: instead of taking the log of a large product, take the sum of the logs).

You'll want to be careful about dictionary sizes when it comes to the Laplace smoothing term: make sure your dictionary size $D$ is equal to the total number of unique words that occur both in the original data used to build the language model _and_ in the text we are evaluating the perplexity of.

As a simple example, if we build our `l_hamilton` model above (again, with `n=3`) and using default settings so that `alpha = 1e-3`, and run in on `papers[0]` (which was written by Hamilton), we get:

    l_hamilton.perplexity(papers[0]) = 12.5877

Using this model, evaluate the mean of the perplexity of the unknown Federalist papers for the language models from each of the three authors (again, using `n=3` and the default of `alpha=1e-3`).  Populate the following variables with the mean perplexities.

In [ ]:
perp_hamilton = None
perp_madison = None
perp_jay = None

### Part C: Generating text

Finally, implement the `sample()` function to generate random samples of text.  Essentially, you want to pick some random starting $n-1$ tuples (you can do this any way you want), then sample according to the model.  Here you should _not_ use any Laplace smoothing, but just sample from the raw underlying counts.

One potential failure case, since we're just using raw counts, is if we generate an n-gram that _only_ occurs at the very end of a document (and so has no following n-gram observed in the data).  In this happens, just generate a new random set of $n-1$ tuples, and continue generating.

We'll be pretty lax in grading here: we're just going to be evaluating the perplexity of the generated text and make sure it's within some very loose bounds.  This is more for the fun of seeing some generated text than for actual concrete evaluation, since it's generating a random sample.  Here's what a sample of 200 words from our Hamilton model looks like (of course all random samples will be different). 

    'erroneous foundation . the authorities essential to the mercy of the body politic against these two legislatures coexisted for ages in two ways : either by actual possession of which , if it cease to regard the servile pliancy of the states which have a salutary and powerful means , by those who appoint them . they are rather a source of delinquency , it would enable the national situation , and consuls , judges of their own immediate aggrandizement ? would she not have been felt by those very maxims and councils which would be mutually questions of property ? and will he not only against the united netherlands with that which has been presumed . the censure attendant on bad measures among a multitude that might have been of a regular and effective system of civil polity had previously established in this state , but upon our neutrality . by thus circumscribing the plan of opposition , and the industrious habits of the trust committed to hands which could not be likely to do anything else . when will the time at which we might soar to a deed for conveying property of the people , were dreaded and detested'

